# **Obtención y preparación de datos**

# OD20. Agrupaciones

Las agrupaciones realizadas con el método de series y dataframes `groupby` son una herramienta un tanto más sofisticada pero extremadamente útil en ciertas circunstancias. También resulta muy útil la creación de tablas dinámicas a partir de un dataframe utilizando el método `pivot_table`. Veamos algunos ejemplos sencillos de estas funciones.

In [ ]:
import numpy as np
import pandas as pd

## <font color='blue'>**Agrupaciones en series**</font>

El método que permite agrupar una serie es `pandas.Series.groupby`. En su sintaxis más básica, requiere el parámetro `by` o el parámetro `level`.

In [ ]:
ventas = pd.Series([2, 4, 1, 6, 2], index = ["A", "B", "C", "A", "C"])
ventas

El parámetro `by` se usa para determinar los grupos. Puede ser una función -que se aplicará a todos los elementos del índice-, un diccionario o una serie -en cuyo caso serán los valores los que determinen los grupos.

Para ver el método `groupby` en funcionamiento con una función que determine los grupos, definamos una que simplemente devuelva la concatenación del texto "Grupo " y el valor que recibe: recordemos que esta función se va a aplicar sobre el índice de la serie, es decir, sobre los elementos "A", "B", etc. La función devolverá, por lo tanto, "Grupo A", "Grupo B", etc. y serán estas etiquetas las que determinen los grupos:



In [ ]:
def grupo(s):
  return("Grupo " + s)

El resultado de la agrupación es un objeto (`SeriesGroupBy` en el caso de las series) que contiene información sobre las agrupaciones pero no es visible. Lo que sí podemos hacer es aplicar a este objeto una función de agregación, por ejemplo el método `mean()` para obtener el valor medio de la serie original para cada uno de los grupos. En este caso tendríamos:

In [ ]:
ventas.groupby(by = grupo).mean()

Hemos comentado que el método puede también recibir como parámetro `by` un diccionario, en cuyo caso serán los valores los que determinen los nombres de los grupos a crear tras mapear las claves del diccionario con las etiquetas de la serie. En nuestro caso, las etiquetas de la serie son "A", "B", etc., por lo que podemos usar el siguiente diccionario para mapear estos valores con los nombres de los grupos a crear: "Producto A", "Producto B", etc. en este ejemplo:

In [ ]:
d = {"A": "Producto A", "B": "Producto B", "C": "Producto C"}
d

Ahora, si aplicamos el método con este diccionario:

In [ ]:
ventas.groupby(by = d).mean()

Vemos que obtenemos un resultado semejante al anterior.

Si, en lugar de hacer uso del parámetro `by`, hacemos uso del parámetro `level`, tendríamos que indicar el nivel del índice según el cual queremos realizar la agrupación (lo que tiene sentido en series con multi índice o índice jerárquico). Si indicamos como nivel el 0, sencillamente estaremos agrupando según las etiquetas de la serie.

In [ ]:
ventas.groupby(level = 0).mean()

## <font color='blue'>**Agrupaciones en dataframes**</font>

El método `pandas.DataFrame.groupby` tiene una funcionalidad semejante a la vista para series, con los condicionantes propios de los dataframes: es necesario indicar el eje que contiene el criterio por el que se va a realizar la agrupación. Comencemos con un ejemplo sencillo.

In [ ]:
ventas = pd.DataFrame({
    "Producto": ["A", "B", "C", "B", "A", "A"],
    "Ventas": [6, 2, 1, 4, 5, 2]
})
ventas

En el caso de los dataframes, el parámetro `by` puede hacer referencia a una función, a un diccionario, a una etiqueta o a una lista de etiquetas. Si pasamos simplemente la etiqueta "Producto" para indicar que la agrupación se realice según los valores de esta columna, tenemos:

In [ ]:
ventas.groupby(by = "Producto").mean()

Si quisiéramos realizar la agrupación por más de una columna, bastaría con pasar como argumento una lista con las etiquetas en cuestión. Por ejemplo, consideremos el siguiente caso en el que tenemos las ventas clasificadas por categoría y producto:



In [ ]:
ventas = pd.DataFrame({
    "Categoría": [1, 2, 1, 1, 2, 1],
    "Producto": ["A", "B", "C", "B", "A", "A"],
    "Ventas": [6, 2, 1, 4, 5, 2]
})
ventas

Si aplicamos ahora el método `groupby` con el argumento `by = ["Categoría", "Producto"]`, tenemos:

In [ ]:
ventas.groupby(by = ["Categoría", "Producto"]).mean()

Este ejemplo tiene demasiados pocos datos para ser significativo, pero aun así es posible ver que el método ha agrupado todas las ventas según la combinación de categoría y producto, y se ha calculado el valor medio. Por ejemplo, hay dos ventas de categoría 1 y producto A, de valores 6 y 2. La media, tal y como se muestra es de 4.

También podríamos usar el parámetro `level`. En el caso de estar trabajando con dataframes con índices no jerárquicos, basta pasar como valor para este argumento el 0 para que la agrupación se realice según las etiquetas del índice. Por ejemplo, consideremos el siguiente dataframe:

In [ ]:
ventas = pd.DataFrame({
    "Ventas": [6, 2, 1, 4, 5, 2]
}, index = ["A", "B", "C", "B", "A", "A"])
ventas

Si ejecutamos el método con el argumento `level = 0`, obtendríamos el siguiente resultado:


In [ ]:
ventas.groupby(level = 0).mean()

## <font color='blue'>**Tablas dinámicas**</font>

Una tabla dinámica (o pivot table en inglés) es una tabla que muestra información resumida extraída de otra tabla. Esta última es un listado de muestras (registros o puntos) con un cierto número de campos o características, por ejemplo:

In [ ]:
df = pd.DataFrame({
    'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
    'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
    'baz': [1, 2, 3, 4, 5, 6],
    'zoo': ['x', 'y', 'z', 'q', 'w', 't']
})
df

Una tabla dinámica va a agrupar información a partir de esta tabla de la siguiente forma:

1. Va a seleccionar una (o más) características para ocupar el índice de filas, de forma que cada valor que tome dicha característica se muestre en una fila
2. Va a seleccionar una (o más) características para ocupar el índice de columnas, de forma que cada valor que tome dicha característica se muestre en una columna
3. Va a seleccionar una (o más) características para ocupar las intersecciones de filas y columnas
4. Al conjunto de registros representados en cada una de esas intersecciones les va a aplicar una función de agregación, que puede ser tan simple como un recuento, cálculo del valor medio, etc.

El método `pandas.DataFrame.pivot_table` crea una tabla dinámica de esta forma a partir de un dataframe. Veamos varios ejemplos comenzando por los más simples:

En el dataframe visto comprobamos que la características "foo" toma dos posibles valores (one y two), y la característica "bar" toma tres (A, B y C). Podríamos mostrar la distribución de la variable "baz" respecto de "foo" y "bar" de la siguiente forma:

In [ ]:
df.pivot_table(index = "foo", columns = "bar", values = "baz")

En este caso, los valores que toma la característica incluida en el parámetro `index` van a distribuirse a lo largo del eje vertical, y los valores que toma la característica incluida en el parámetro `columns` van a distribuirse a lo largo del eje horizontal. Los valores que toma la variable incluida en el parámetro `values` van a la intersección de filas y columnas, aplicándoseles una cierta función de agregación que, por defecto, es `np.mean` (cálculo del valor medio). El ejemplo mostrado es muy pequeño y para cada intersección de filas y columnas solo hay un registro, de forma que el valor medio del valor contenido en la columna baz de cada registro coincide con el mismo valor. Por ejemplo, la intersección de foo = one y bar = A representa un conjunto de registros del dataframe que, en nuestro caso, se limita a un único registro (el registro con índice 0) en el que el valor de baz es 1, y su valor medio es 1.

Podemos aplicar otra función de agregación utilizando el parámetro `aggfunc`.

In [ ]:
df.pivot_table(index = "foo", columns = "bar", values = "baz", aggfunc = "count")

En este ejemplo hemos contado el número de registros representados en cada intersección.

Es posible aplicar más de una función de agregación a los datos. En el siguiente ejemplo aplicamos tanto la función de cálculo del valor medio como el recuento:

In [ ]:
df.pivot_table(index = "foo", columns = "bar", values = "baz", aggfunc = [np.mean, "count"])

Como puede comprobarse, pandas crea un conjunto de columnas diferente para cada función de agregación.

Hagamos algunos ejemplos con un dataset un poco más rico en contenido, por ejemplo el dataset del Titanic:

In [ ]:
import seaborn as sns
titanic = sns.load_dataset("titanic")
titanic.head(5)

Mostremos el valor medio de la característica survived (es decir, el porcentaje de los que sobrevivieron) desglosando la tabla por sexo y clase:

In [ ]:
titanic.pivot_table(index = "sex", columns = "class", values = "survived")

Si llevamos dos (o más) campos a index, los valores que tome el primero de ellos se desglosa a su vez según los valores que tome el segundo. Por ejemplo, podemos repetir el ejercicio anterior desglosando las filas por sexo y puerto de embarque:

In [ ]:
titanic.pivot_table(index = ["sex", "embarked"], columns = "class", values = "survived")

De forma semejante, si llevamos dos (o más) campos a columns, los valores que tome el primero de ellos se desglosa a su vez según los valores que tome el segundo. En el siguiente ejemplo queremos analizar el valor medio de la edad de los pasajeros por clase (en filas) y por sexo y si viajaba o no solo (por columnas):

In [ ]:
titanic.pivot_table(index = "class", columns = ["sex", "alone"], values = "age")

Por último, si llevamos dos (o más) campos a values, pandas va a crear un conjunto de columnas para cada uno de dichos campos:

In [ ]:
titanic.pivot_table(index = "sex", columns = "class", values = ["survived", "age"])

Este método incluye también parámetros que permite rellenar los valores nulos (`fill_value`) y añadir subtotales de filas y columnas (`margins`).

### <font color='green'>Actividad 1</font>

Se tiene un conjunto de restaurantes en ciudades de Chile, en las que se tiene cada ciudad y el tipo de cocina en cada una.



```
data_restaurantes = {
    'ciudades': ['Valparaíso','Valparaíso','Valparaíso','Valparaíso','Valparaíso','Valparaíso','Santiago','Santiago','Santiago','Santiago','Santiago','Punta Arenas','Punta Arenas','Punta Arenas'],
    'culinaria': ['Chorrillana','Chorrillana','Charquicán','Pulmay','Tallarines','Chorrillana','Tallarines','Charquicán','Porotos','Chorrillana','Porotos','Porotos','Tallarines','Charquicán']
}

restaurantes_dataframe_pares = pd.DataFrame(data_restaurantes)
restaurantes_dataframe_pares
```

1. Generar una tabla que permita contar la presencia de cada tipo de cocina en cada ciudad.
2. Genera una tabla sólo para la ciudad de Valparaíso.



In [ ]:
# Tu código aquí ...


<font color='green'>Fin Actividad 1</font>


### <font color='green'>Actividad 2</font>

Eres el analista de datos de una cadena de tiendas que vende diferentes tipos de productos electrónicos. Tienes un conjunto de datos que contiene información sobre las ventas de diferentes productos en varias sucursales de la cadena durante un mes determinado. Tu tarea es analizar y resumir esta información para identificar patrones y tendencias.

Genera un DataFrame simulado con los siguientes datos:

```
import pandas as pd
import numpy as np

np.random.seed(0)

sucursales = ['Centro', 'Norte', 'Sur', 'Este', 'Oeste']
productos = ['Smartphone', 'Laptop', 'Audífonos', 'Cargador', 'Tablet']

data = {
    'Sucursal': np.random.choice(sucursales, 500),
    'Producto': np.random.choice(productos, 500),
    'Ventas (unidades)': np.random.randint(1, 50, 500),
    'Ingreso ($)': np.random.randint(10, 1000, 500)
}

df = pd.DataFrame(data)
```

1. Agrupa el DataFrame por 'Sucursal' y calcula el ingreso total por sucursal.
2. Determina cuál es el producto más vendido en términos de unidades para cada sucursal.
3. Calcula el precio promedio de venta (Ingreso $ / Ventas unidades) para cada producto.
4. Agrupa por 'Producto' y 'Sucursal' y calcula el total de unidades vendidas. 5. ¿Hay algún producto que se venda particularmente bien o mal en alguna sucursal específica?
6. Encuentra la sucursal que tiene el mayor número de ventas (unidades) y el producto más vendido en esa sucursal.



In [ ]:
# Tu código aquí ...


<font color='green'>Fin Actividad 2</font>


### <font color='green'>Actividad 3</font>

Trabajas como analista de datos en una plataforma de cursos en línea. Se te ha proporcionado un conjunto de datos que contiene la actividad de los usuarios en diferentes cursos. Los datos incluyen el día, el curso en el que estaban activos, la duración de la actividad y el tipo de actividad (por ejemplo, "Ver Video", "Participar en Foro", "Realizar Examen").

Tu tarea es utilizar tablas dinámicas para analizar y resumir esta actividad y obtener insights sobre el comportamiento de los usuarios.

Genera un DataFrame simulado con los siguientes datos:

```
import pandas as pd
import numpy as np

np.random.seed(123)

dias = pd.date_range(start="2022-01-01", end="2022-01-31", freq='D')
cursos = ['Python Básico', 'Data Science Avanzado', 'Machine Learning', 'Web Development', 'Diseño Gráfico']
actividades = ['Ver Video', 'Participar en Foro', 'Realizar Examen']

data = {
    'Fecha': np.random.choice(dias, 1000),
    'Curso': np.random.choice(cursos, 1000),
    'Actividad': np.random.choice(actividades, 1000),
    'Duración (min)': np.random.randint(5, 120, 1000)
}

df = pd.DataFrame(data)
```

1. Crea una tabla dinámica que muestre el tiempo total (suma de la duración) que los usuarios pasaron en cada actividad, desglosado por curso.
2. Muestra el promedio de tiempo que los usuarios pasan en las actividades, pero esta vez desglosado por día del mes.
3. Encuentra qué curso tiene la mayor participación en foros en promedio durante los fines de semana (sábados y domingos).
4. ¿Cuál es la actividad principal (en términos de tiempo total) para cada curso?


In [ ]:
# Tu código aquí ...


<font color='green'>Fin Actividad 3</font>
